In [4]:
pip install twilio opencv-python ultralytics

  Using cached opencv-python-4.10.0.82.tar.gz (95.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.3 MB/s eta 0:00:00
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached filelock-3.14.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached sympy-1.12.1-py3-none-any.whl.metadata (12 kB)
  Using cached fsspec-2024.6.0-py3-none-any.whl.metadata (11 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.6/780.6 kB 6.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 MB 7.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.7 MB/s eta 0:00:00a 0:00:01
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
Using cached filelock-3.14.0-py3-none-any.whl (12 kB)
Using cached fsspec-2024.6.0-

In [2]:
from twilio.rest import Client

account_sid = input("account_sid : ")
auth_token = input("auth_token : ")
client = Client(account_sid, auth_token)


In [3]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
#model = YOLO('yolov8s.pt')
model = YOLO('yolov8n.pt')

# 동영상 파일 사용시
# video_path = "path/to/your/video/file.mp4"
# cap = cv2.VideoCapture(video_path)

CONFIDENCE_THRESHOLD = 0.3

# webcam 사용시 
cap = cv2.VideoCapture(0)

x_offset = 480
y_offset = 0
cnt = 0
s_img = cv2.imread("wait2.png", -1)
y1, y2 = y_offset, y_offset + s_img.shape[0]
x1, x2 = x_offset, x_offset + s_img.shape[1]
alpha_s = s_img[:, :, 3] / 255.0
alpha_l = 1.0 - alpha_s
check_yn = False

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame)
        
        # Visualize the results on the frame
        annotated_frame = results[0].plot()
        
        for data in results[0].boxes.data.tolist():
            confidence = float(data[4])
            if confidence < CONFIDENCE_THRESHOLD :
                continue
            label = int(data[5])
            if label == 67 :
                cnt+=1
                print("cell-phone detected! cnt : ", cnt)

            if not check_yn and cnt  >= 20 :
                for c in range(0, 3):
                    annotated_frame[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] +
                                      alpha_l * annotated_frame[y1:y2, x1:x2, c])
                print("voice phising??")

        
        if not check_yn and cv2.waitKey(1) & 0xFF == ord("1"):
            check_yn = True
            
            message = client.messages.create(
                to="+821031061041",
                from_="+12568125366",
                body="신한은행 성수동지점 ATM기에 보이스피싱 의심사례 발생!")
            
            print(message.sid)
        elif not check_yn and cv2.waitKey(1) & 0xFF == ord("2"):
            check_yn = True
        
        # Display the annotated frame
        cv2.imshow("YOLOv8 Inference", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
            
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'cv2'

In [ ]:
a